In [ ]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Callable, Union

In [ ]:
class Expression(ABC):

    def __invert__(self) -> 'Not':
        return Not(self)

    def __and__(self, other: 'Expression') -> 'And':
        return And(self, other)

    def __or__(self, other: 'Expression') -> 'Or':
        return Or(self, other)

    def __le__(self, other: 'Expression') -> 'Implication':
        return Implication(self, other)

    def __ge__(self, other: 'Expression') -> 'RevImplication':
        return RevImplication(self, other)

    def __add__(self, other: 'Expression') -> 'Equivalence':
        """To not override ==, + will be logical equivalence"""
        return Equivalence(self, other)

    def __xor__(self, other: 'Expression') -> 'Xor':
        return Xor(self, other)

    @abstractmethod
    def simplify(self) -> 'Expression':
        ...


@dataclass(frozen=True)
class Literal(Expression):
    name: str

    def __str__(self):
        return self.name

    def __repr__(self):
        return f'Literal({self.name})'

    def simplify(self):
        return self


_0 = Literal('0')
_1 = Literal('1')


@dataclass(frozen=True)
class Not(Expression):
    expr: Expression

    def __str__(self):
        template = '~%s' \
            if isinstance(self.expr, Literal | Not) \
            else '~(%s)'
        return template % self.expr

    def __repr__(self):
        return f'Not({repr(self.expr)})'

    def simplify(self):
        result = self.expr.simplify()
        reverse = False
        if isinstance(result, Not):
            result = result.expr
            reverse = True
        if result == _0:
            return (_1, _0)[reverse]
        if result == _1:
            return (_0, _1)[reverse]
        return result if reverse else Not(result)


@dataclass(frozen=True)
class And(Expression):
    left_expr: Expression
    right_expr: Expression

    def __str__(self):
        left_template = '%s' \
            if isinstance(self.left_expr, Literal | Not) \
            else '(%s)'
        right_template = '%s' \
            if isinstance(self.right_expr, Literal | Not) \
            else '(%s)'
        template = left_template + ' & ' + right_template
        return template % (self.left_expr, self.right_expr)

    def __repr__(self):
        return f'And({repr(self.left_expr)}, {repr(self.right_expr)})'

    def simplify(self):
        left_result = self.left_expr.simplify()
        right_result = self.right_expr.simplify()

        if _0 in (left_result, right_result):
            return _0
        if left_result == _1:
            return right_result
        if right_result == _1:
            return left_result
        return And(left_result, right_result)


@dataclass(frozen=True)
class Or(Expression):
    left_expr: Expression
    right_expr: Expression

    def __str__(self):
        left_template = '%s' \
            if isinstance(self.left_expr, Literal | Not) \
            else '(%s)'
        right_template = '%s' \
            if isinstance(self.right_expr, Literal | Not) \
            else '(%s)'
        template = left_template + ' | ' + right_template
        return template % (self.left_expr, self.right_expr)

    def __repr__(self):
        return f'Or({repr(self.left_expr)}, {repr(self.right_expr)})'

    def simplify(self):
        left_result = self.left_expr.simplify()
        right_result = self.right_expr.simplify()
        if _1 in (left_result, right_result):
            return _1
        if left_result == _0:
            return right_result
        if right_result == _0:
            return left_result
        return Or(left_result, right_result)


@dataclass(frozen=True)
class Implication(Expression):
    left_expr: Expression
    right_expr: Expression

    def __str__(self):
        left_template = '%s' \
            if isinstance(self.left_expr, Literal | Not) \
            else '(%s)'
        right_template = '%s' \
            if isinstance(self.right_expr, Literal | Not) \
            else '(%s)'
        template = left_template + ' <= ' + right_template
        return template % (self.left_expr, self.right_expr)

    def __repr__(self):
        return f'Implication({repr(self.left_expr)}, {repr(self.right_expr)})'

    def simplify(self):
        left_result = self.left_expr.simplify()
        right_result = self.right_expr.simplify()
        if left_result == _0:
            return _1
        if left_result == _1 and right_result == _0:
            return _0
        return Implication(left_result, right_result)


@dataclass(frozen=True)
class RevImplication(Expression):
    left_expr: Expression
    right_expr: Expression

    def __str__(self):
        left_template = '%s' \
            if isinstance(self.left_expr, Literal | Not) \
            else '(%s)'
        right_template = '%s' \
            if isinstance(self.right_expr, Literal | Not) \
            else '(%s)'
        template = left_template + ' >= ' + right_template
        return template % (self.left_expr, self.right_expr)

    def __repr__(self):
        return \
            f'RevImplication({repr(self.left_expr)}, {repr(self.right_expr)})'

    def simplify(self):
        left_result = self.left_expr.simplify()
        right_result = self.right_expr.simplify()
        if right_result == _0:
            return _1
        if right_result == _1 and left_result == _0:
            return _0
        return RevImplication(left_result, right_result)


@dataclass(frozen=True)
class Equivalence(Expression):
    left_expr: Expression
    right_expr: Expression

    def __str__(self):
        left_template = '%s' \
            if isinstance(self.left_expr, Literal | Not) \
            else '(%s)'
        right_template = '%s' \
            if isinstance(self.right_expr, Literal | Not) \
            else '(%s)'
        template = left_template + ' + ' + right_template
        return template % (self.left_expr, self.right_expr)

    def __repr__(self):
        return f'Equivalence({repr(self.left_expr)}, {repr(self.right_expr)})'

    def simplify(self):
        left_result = self.left_expr.simplify()
        right_result = self.right_expr.simplify()
        if left_result == right_result:
            return _1
        return self


@dataclass(frozen=True)
class Xor(Expression):
    left_expr: Expression
    right_expr: Expression

    def __str__(self):
        left_template = '%s' \
            if isinstance(self.left_expr, Literal | Not) \
            else '(%s)'
        right_template = '%s' \
            if isinstance(self.right_expr, Literal | Not) \
            else '(%s)'
        template = left_template + ' ^ ' + right_template
        return template % (self.left_expr, self.right_expr)

    def __repr__(self):
        return f'Xor({repr(self.left_expr)}, {repr(self.right_expr)})'

    def simplify(self):
        left_result = self.left_expr.simplify()
        right_result = self.right_expr.simplify()
        if left_result == right_result:
            return _0
        return self

In [ ]:
class Infix:

    def __init__(self, func: Callable,
                 value: Union['Expression', None] = None):
        self.func = func
        self.value = value

    def __gt__(self, other: 'Expression'):
        if self.value is None:
            self.value = other
            return self
        assert self.value is not None
        value = self.value
        self.value = None
        return self.func(value, other)


equiv = Infix(lambda x, y: Equivalence(x, y), None)

In [ ]:
for lit in 'ABCDE':
    exec(f'{lit} = Literal("{lit}")')

In [ ]:
e1 = A&B|C&A + A | C
print(e1)
e1

In [ ]:
e2 = e1.simplify()
print(e2)
e2

In [ ]:
# e3 = ~A & B | C ^ D <= E %equiv% ~A | B
# e4 = ((((~A) & B) | (C ^ D)) <= E) %equiv% (~A | B)
e3 = (~A & B | C ^ D <= E) <equiv> ~A | B
e4 = ((((~A) & B) | (C ^ D)) <= E) + (~A | B)
print(e3)
print(e4)